# Convert a PyTorch Model to CoreML formats

In [2]:
%pip install transformers
%pip install -U coremltools

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.3
    Uninstalling protobuf-4.24.3:
      Successfully uninstalled protobuf-4.24.3
Note: you may need to restart the kernel to use updated packages.


## PyTorch Conversion Workflow

In [2]:
import torch
import torchvision

In [12]:
mobilenetv2 = torchvision.models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')
mobilenetv2.eval()

# Trace the model with random data
example_input = torch.rand(1, 3, 224, 224) # (B,C,S,S)
traced_model = torch.jit.trace(mobilenetv2, example_input)
x = traced_model(example_input)
print(x.shape)


torch.Size([1, 1000])


### Convert it to CoreML

In [14]:
import coremltools as ct

modelprog = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
)

Running MIL default pipeline:   0%|          | 0/71 [00:00<?, ? passes/s]/opt/homebrew/lib/python3.11/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:239: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1488.88 passes/s]


In [15]:
# save it from memory to disk
modelprog.save("newmodel.mlpackage")

In [20]:
# convert it to a neural network format

modelml = ct.convert(
    traced_model,
    convert_to="neuralnetwork",
    inputs=[ct.TensorType(shape=example_input.shape)]
)

modelml.save("newmodel.mlmodel")

Running MIL default pipeline:   0%|          | 0/69 [00:00<?, ? passes/s]/opt/homebrew/lib/python3.11/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:239: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 495/495 [00:00<00:00, 3071.08 ops/s]
